<a href="https://colab.research.google.com/github/smithkanzariya/Fake-News-Classifier/blob/master/FakeNewsClassifierUsingBidirectionalLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
Dataset  = pd.read_csv("/content/drive/My Drive/train.csv")
Dataset.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


In [2]:
#Drop NAN Values
Dataset = Dataset.dropna()

In [3]:
#Get the independent Feature
X = Dataset.drop("label", axis = 1)
X.shape
Y = Dataset["label"]
Y.shape

(18285,)

In [4]:
Y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [5]:
X.shape


(18285, 4)

In [6]:
import tensorflow as tf 

In [7]:
tf.__version__

'2.8.0'

In [8]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import  Bidirectional

In [9]:
#Vocabulary size
voc_size = 5000

Onehot Representaion

In [10]:
messages = X.copy()
messages.reset_index(inplace = True)
messages["title"][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [11]:
import nltk
import re
from nltk.corpus import stopwords

In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
#Data Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  print(i)
  review = re.sub("[^a-zA-Z]", " ", messages["title"][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if word not in stopwords.words("english")]
  review = " ".join(review)
  corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [14]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [15]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[4855, 4500, 3494, 1056, 3431, 4857, 1801, 2366, 3304, 4613],
 [1327, 2602, 1389, 1342, 1088, 3714, 2862],
 [1294, 1202, 3590, 2695],
 [3317, 2652, 1458, 4762, 2575, 1351],
 [1159, 1088, 2367, 1146, 705, 4378, 1088, 3345, 1803, 115],
 [4457,
  85,
  4127,
  4209,
  884,
  3675,
  3473,
  3894,
  1724,
  4777,
  4672,
  1482,
  4368,
  2678,
  2862],
 [93, 346, 3313, 833, 3784, 1354, 4445, 3352, 1101, 1920, 115],
 [2092, 1025, 1929, 3955, 1699, 2716, 3675, 409, 1101, 1920, 115],
 [3614, 2555, 3045, 4026, 1150, 1329, 2669, 3589, 3675, 1865],
 [4539, 4021, 2658, 1870, 864, 562, 2459, 3461],
 [4950, 3189, 1856, 2561, 4550, 1842, 4830, 1144, 3586, 3612, 2758],
 [4762, 3578, 3431, 1329, 3675, 1699],
 [3867, 1285, 107, 4187, 2977, 2484, 1768, 4146, 1345],
 [3026, 543, 462, 3814, 4954, 2502, 3671, 1101, 1920, 115],
 [2339, 2560, 4309, 2792, 2559, 1101, 1920, 115],
 [4359, 4096, 2929, 2778, 239, 4385, 785, 1284, 2014, 401],
 [4484, 4219, 2602],
 [3346, 2586, 1070, 2615, 3675, 3300, 1410, 2862]

In [16]:
longest = max(onehot_repr, key = len)
print(len(longest))

47


Embedding Rpresentation

In [17]:
sent_lenght = 20
embedded_docs = pad_sequences(onehot_repr, padding="pre", maxlen= sent_lenght)
print(embedded_docs)

[[   0    0    0 ... 2366 3304 4613]
 [   0    0    0 ... 1088 3714 2862]
 [   0    0    0 ... 1202 3590 2695]
 ...
 [   0    0    0 ... 1101 1920  115]
 [   0    0    0 ... 1663 1780 4986]
 [   0    0    0 ... 3017  116 1004]]


In [18]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4855,
       4500, 3494, 1056, 3431, 4857, 1801, 2366, 3304, 4613], dtype=int32)

In [19]:
from tensorflow.keras.layers import Dropout
#Creating model
embedding_vector_feature = 40
model1 = Sequential()
model1.add(Embedding(voc_size, embedding_vector_feature, input_length = sent_lenght))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dense(1, activation= "sigmoid"))
model1.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
print(model1.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
len(embedded_docs), Y.shape

(18285, (18285,))

In [21]:
import numpy as np
X_final = np.array(embedded_docs)
Y_final = np.array(Y)

In [22]:
X_final.shape, Y_final.shape

((18285, 20), (18285,))

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_final, Y_final, test_size = 0.33, random_state = 42)

Model Training

In [24]:
#Training Time
model1.fit(X_train, Y_train, validation_data = (X_test, Y_test), epochs = 10, batch_size = 64)

Epoch 1/10
192/192 [==============================] - 14s 54ms/step - loss: 0.2918 - accuracy: 0.8611 - val_loss: 0.2001 - val_accuracy: 0.9170
Epoch 2/10
192/192 [==============================] - 9s 47ms/step - loss: 0.1364 - accuracy: 0.9483 - val_loss: 0.2245 - val_accuracy: 0.9180
Epoch 3/10
192/192 [==============================] - 9s 49ms/step - loss: 0.0888 - accuracy: 0.9684 - val_loss: 0.2735 - val_accuracy: 0.9105
Epoch 4/10
192/192 [==============================] - 10s 50ms/step - loss: 0.0559 - accuracy: 0.9797 - val_loss: 0.3252 - val_accuracy: 0.9147
Epoch 5/10
192/192 [==============================] - 10s 50ms/step - loss: 0.0368 - accuracy: 0.9873 - val_loss: 0.3429 - val_accuracy: 0.9157
Epoch 6/10
192/192 [==============================] - 10s 51ms/step - loss: 0.0239 - accuracy: 0.9910 - val_loss: 0.4448 - val_accuracy: 0.9112
Epoch 7/10
192/192 [==============================] - 10s 50ms/step - loss: 0.0180 - accuracy: 0.9948 - val_loss: 0.4071 - val_accuracy: 0

Performence Metrics and Accuracy 

In [26]:
Y_pred1 = model1.predict(X_test)

In [29]:
classes_x=np.argmax(Y_pred1,axis=1)

In [27]:
from sklearn.metrics import confusion_matrix

In [30]:
confusion_matrix(Y_test, classes_x)

array([[3419,    0],
       [2616,    0]])

In [32]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, classes_x)

0.5665285832642917

In [34]:
from sklearn.metrics import  classification_report
print(classification_report(Y_test, classes_x))

              precision    recall  f1-score   support

           0       0.57      1.00      0.72      3419
           1       0.00      0.00      0.00      2616

    accuracy                           0.57      6035
   macro avg       0.28      0.50      0.36      6035
weighted avg       0.32      0.57      0.41      6035



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
